In [1]:
import pandas as pd
import numpy as np

import math
import scipy.optimize as opt
import random
random.seed(108)


In [2]:
# Data: Close
db = pd.read_excel('Data.xlsx', index_col = "Date")

# MSD

In [3]:
db.index = pd.to_datetime(db.index)
MSD_df = db['2006':'2013']
MSD_df = (MSD_df-MSD_df.mean())/MSD_df.std() #change rolling
MSD_df = MSD_df.dropna(axis = 1) #drop nan

In [4]:
target = {}
for i in range(0,len(MSD_df.columns)-1):
    for j in range(i+1,len(MSD_df.columns)):
        tmp = ((MSD_df.iloc[:,i] - MSD_df.iloc[:,j])**2).sum()
        target[str(MSD_df.columns[i])+','+str(MSD_df.columns[j])] = tmp

In [5]:
target_df = pd.DataFrame.from_dict(target, orient='index')
target_df = target_df.sort_values(by = 0)[0:10]
target_df

,0
"DIS,TRV",106.019533
"IBM,MCD",174.553811
"AAPL,IBM",183.653680
"CVX,KO",205.031258
"IBM,KO",225.295065
"DIS,HD",264.348677
"HD,TRV",282.786437
"PEP,PG",298.596370
"DIS,MMM",300.548727
"GOOGL,TRV",304.718140


In [6]:
pair1_df_train = db.loc["2006":"2013",target_df.index[5].split(",")].pct_change(1).dropna()
pair1_df_test = db.loc["2014":"2014",target_df.index[5].split(",")].pct_change(1).dropna()
pair1_df_train

,DIS,HD
Date,,
2006-01-04,-0.016803,-0.018429
2006-01-05,0.017507,-0.009881
2006-01-06,0.013519,0.007485
2006-01-09,0.010509,0.010896
2006-01-10,0.012800,0.024008
...,...,...
2013-12-24,0.007778,0.000993
2013-12-26,0.010427,0.011285
2013-12-27,-0.003618,0.001104


In [7]:
# ytlist = pair1_df_train.iloc[:,0] - pair1_df_train.iloc[:,1]

In [8]:
from scipy.stats import t

# ytlist = [] #用來收集過去的yt們，算z_td、s_z時會用到

# #Table1 參數
# fi_1_0 = 0.1046
# fi_1_1 = 0.3886
# fi_2_0 = 0.0961
# fi_2_1 = -0.2403
# alpha_1_0 = 0.1639
# alpha_1_1 = 0.2132
# beta_1_1 = 0.6631
# alpha_2_0 = -0.1035
# alpha_2_1 = -0.1063
# beta_2_1 = -0.1751
# v = 7
# gamma = 5.5563
# c_1 = -0.3569
# c_2 = 0.3136
# d = 1
# z_td = yt[len(yt)-d:-1] #t-d
# s_z = std(z_td)

# Fdis = (1+np.exp(-gamma*(z_td - c_1)(z_td - c_2))/s_z)**(-1)
# h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
# h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
# miu_1 = fi_1_0 + fi_1_1*y_t1
# miu_2 = fi_2_0 + fi_2_1*y_t1
# h_t = h_1_t + Fdis*h_2_t
# epsilon = t.rvs(v, size=1) #t-distribution
# a_t = h_t**0.5*epsilon #error term
# y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH


In [9]:
z_td = pair1_df_train.iloc[0:30,0]-pair1_df_train.iloc[0:30,1]
z_td

Date
2006-01-04    0.001625
2006-01-05    0.027389
2006-01-06    0.006034
2006-01-09   -0.000387
2006-01-10   -0.011208
2006-01-11   -0.018033
2006-01-12    0.012443
2006-01-13    0.017381
2006-01-17   -0.004339
2006-01-18   -0.015031
2006-01-19    0.056067
2006-01-20    0.007072
2006-01-23    0.004173
2006-01-24   -0.000732
2006-01-25   -0.009543
2006-01-26   -0.024656
2006-01-27    0.009901
2006-01-30    0.015151
2006-01-31   -0.019642
2006-02-01    0.006311
2006-02-02   -0.000016
2006-02-03   -0.000082
2006-02-06    0.006288
2006-02-07    0.088957
2006-02-08   -0.006452
2006-02-09   -0.001089
2006-02-10   -0.005850
2006-02-13   -0.022359
2006-02-14   -0.023146
2006-02-15   -0.004865
dtype: float64

In [10]:
# fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, v, gamma, c_1, c_2 = np.array([0.1046, 0.3886, 0.0961, -0.2403, 0.1639, 0.2132, 0.6631, -0.1035, -0.1063, -0.1751, 7.1122, 5.5563, -0.3569, 0.3136])
# tmp = 30
# z_td = pair1_df_train.iloc[0:tmp,0] - pair1_df_train.iloc[0:tmp,1]
# s_z = z_td.std()   
# Fdis = (1+np.exp(-gamma*(z_td - c_1)*(z_td - c_2)/s_z))**(-1)
# Fdis

In [11]:
def fun_MLE(x):
    fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, gamma, c_1, c_2= x
    # Define h_t1
    tmp = 30
    target = 0
    z_td = pair1_df_train.iloc[0:tmp,0]-pair1_df_train.iloc[0:tmp,1]
    z_td = z_td.to_numpy()
    s_z = z_td.std()
    Fdis = (1+np.exp(-gamma*(z_td[-1] - c_1)*(z_td[-1] - c_2)/s_z))**(-1)
    h_1_t = pair1_df_train.iloc[0:tmp,0].std()
    h_2_t = pair1_df_train.iloc[0:tmp,1].std()
    h_t1 = h_1_t + Fdis*h_2_t
    epsilon = random.gauss(mu = 0, sigma = 1)
    a_t1 = h_t1**0.5*epsilon #error term


    y_t1 = pair1_df_train.iloc[tmp-1,0] - pair1_df_train.iloc[tmp-1,1]
    # Define sigma, real_y_t, y_t
    for i in range(tmp,len(pair1_df_train)):
        z_td = pair1_df_train.iloc[0:i,0]-pair1_df_train.iloc[0:i,1]
        z_td = z_td.to_numpy()
        s_z = z_td.std()
        real_y_t = pair1_df_train.iloc[i,0] - pair1_df_train.iloc[i,1]
        # real_y_t = real_y_t.to_numpy()
        h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
        h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
        miu_1 = fi_1_0 + fi_1_1*y_t1
        miu_2 = fi_2_0 + fi_2_1*y_t1
        h_t = h_1_t + Fdis*h_2_t
        epsilon = random.gauss(mu = 0, sigma = 1)
        a_t = h_t**0.5*epsilon #error term
        y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH

        h_t1 = h_t
        a_t1 = a_t
        y_t1 = real_y_t

        sigma = h_t
        target += np.log((2*np.pi*sigma**2)**(-0.5)*np.exp(-(real_y_t-y_t)**2/(2*sigma**2))) #MLE
        # print(target)
    target = -target
    
    return target

In [12]:
#MLE
tmp = 1e-9*4
initial = np.array([0.1068, 0.3838, 0.0940, -0.2366, 0.1649, 0.2062, 0.6622, -0.1045, -0.0981, -0.1827, 4.9262, -0.3516, 0.3068]) 
b1 = ((pair1_df_train.iloc[0:30,0] - pair1_df_train.iloc[0:30,1]).std())**2*10000
b2 = 1
b3 = 1.1
constraints = ( {'type':'ineq','fun':lambda x: x[4]+tmp},
                {'type':'ineq','fun':lambda x: x[5]+tmp},
                {'type':'ineq','fun':lambda x: x[6]+tmp},
                {'type':'ineq','fun':lambda x: x[5]+x[8]+tmp},
                {'type':'ineq','fun':lambda x: x[6]+x[9]+tmp},
                {'type':'ineq','fun':lambda x:-x[5]-0.5*x[8]-x[6]-0.5*x[9]+1+tmp},
                {'type':'ineq','fun':lambda x:-x[4]+b1+tmp},     #b1
                {'type':'ineq','fun':lambda x:-x[6]+b2+tmp},     #b2 = 1
                {'type':'ineq','fun':lambda x:-x[5]-x[6]+b3+tmp},#b3 = 1.1
                {'type':'ineq','fun':lambda x: x[12]-x[11]+tmp} 
                  )
bound = (-0.9,0.9)
# bounds = tuple(bound for asset in range(len(initial)))
bounds = ((-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-5, 5),
 (-0.9, 0.9),
 (-0.9, 0.9))
minimum = opt.minimize(fun_MLE, initial, constraints = constraints, method = 'trust-constr', bounds = bounds) #bounds: 0.9
minimum

/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:186: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 2125
      cg_stop_cond: 2
            constr: [array([0.20447194]), array([0.33255101]), array([0.51453841]), array([0.31908845]), array([0.30664506]), array([0.26358855]), array([5.1115791]), array([0.48546159]), array([0.25291059]), array([0.60009635]), array([ 2.61213803e-01,  1.46268625e-01,  4.08260791e-03, -1.92053336e-01,
        2.04471932e-01,  3.32551006e-01,  5.14538410e-01, -1.90969216e-01,
       -1.34625612e-02, -2.07893358e-01,  4.52987703e+00, -3.43630713e-01,
        2.56465630e-01])]
       constr_nfev: [5992, 5992, 5992, 5992, 5992, 5992, 5992, 5992, 5992, 5992, 0]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 2710.8987154960632
               fun: 2961.946370507688
              grad: array([ 4.43349736e+09,  8.15480570e+09,  2.8622608

In [13]:
minimum.x

array([ 2.61213803e-01,  1.46268625e-01,  4.08260791e-03, -1.92053336e-01,
        2.04471932e-01,  3.32551006e-01,  5.14538410e-01, -1.90969216e-01,
       -1.34625612e-02, -2.07893358e-01,  4.52987703e+00, -3.43630713e-01,
        2.56465630e-01])

In [14]:
np.savetxt(str(target_df.index[5].split(",")[0])+'_'+str(target_df.index[5].split(",")[1])+'.txt', minimum.x, delimiter=',' )

# DIS, TRV

In [15]:
minimum.x

array([ 2.61213803e-01,  1.46268625e-01,  4.08260791e-03, -1.92053336e-01,
        2.04471932e-01,  3.32551006e-01,  5.14538410e-01, -1.90969216e-01,
       -1.34625612e-02, -2.07893358e-01,  4.52987703e+00, -3.43630713e-01,
        2.56465630e-01])

# Trade

In [16]:
df_return = db.pct_change()

In [17]:
global results

In [18]:
results = {'signals':[],
           'returns':[],
           'A_SoldPrices':[],
           'A_BoughtPrices':[],
           'B_SoldPrices':[],
           'B_BoughtPrices':[]
           }

In [19]:
# Input1 : Real return spread
# Input2 : Estimated return spread
# Output : Gain, Trading Signal

y1 = df_return.iloc[-1,0] 
y2 = df_return.iloc[-1,1]
p1 = db.iloc[-1:,0] #A Price
p2 = db.iloc[-1,1] #B Price

#First trade
i = 0

#signal
tmp0 = 0
tmp1 = 1
tmp2 = -1

#First trade
if i == 0:
    if (y1 > y2):
        signal = tmp1
        results['A_SoldPrices'].append(p1)
        results['B_BoughtPrices'].append(p2)
        i = 1
        tmp = 0
    
    elif (y1 < y2):  
        signal = tmp2
        results['A_BoughtPrices'].append(p1)
        results['B_SoldPrices'].append(p2) 
        i = 1
        tmp = 0
    else:
        signal = tmp0
        i = 0
        tmp = 1

elif i == 1:
    # After first trade
    if (y1 > y2) & (results['signals'][-1] == tmp1) :
        signal = tmp1
        results['A_SoldPrices'].append(y1)
        results['B_BoughtPrices'].append(y2)
        tmp = -np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])+np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1])
  
    elif (y1 <y2) & (results['signals'][-1] == tmp2):  
        signal = tmp2
        results['A_BoughtPrices'].append(y1)
        results['B_SoldPrices'].append(y2)
        tmp = np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])-np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1]) 

    else:
        signal = tmp0
        tmp = 0


results['signals'].append(signal)
results['returns'].append(tmp)